In [7]:
import sys
sys.path.append("../src") 
sys.path.append("../data") 
import SDE_class as sde
import importlib
importlib.reload(sde)
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import util as util
importlib.reload(util)
import math

# 独立増分の確認

$$X_t-X_s \amalg X_s$$
$$X_t-X_s \sim N(0,t-s)$$

In [40]:
term=1
step=0.001
div=term/step
init=np.array([0.])
repeat_time=100

In [41]:
mymodel = sde.SDE_Markov(mymat=np.array([[0.]]), myvar =np.array([[1.]]),myinit=np.array([1.]),myscale=1.,myterm=term,step_size=step)

In [48]:
time_t=0.05
time_s=0.03

In [49]:
time_t_box=np.zeros(repeat_time)
time_s_box=np.zeros(repeat_time)
time_ts_box=np.zeros(repeat_time)

In [50]:
for k in range(repeat_time):
    many_step = mymodel.many_step_test(now_position=init)
    time_t_box[k]=many_step[0,time_t*div]
    time_s_box[k]=many_step[0,time_s*div]
    time_ts_box[k]=many_step[0,time_t*div]-many_step[0,time_s*div]
var_s=np.var(time_s_box)
var_ts=np.var(time_ts_box)
print(var_s)
print(var_ts)

/Users/tamamoon/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  app.launch_new_instance()
/Users/tamamoon/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/tamamoon/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


33.0144034279
17.8551229629


In [37]:
repeat_time=100000
test=np.zeros(repeat_time)

In [39]:
for k in range(repeat_time):
    test[k]=mymodel.one_step_test(init)
np.var(test)

1.0052687091592607

In [2]:
#int 整数
#float 少数

In [8]:
term=1
step=0.01
init=np.array([0.])

In [9]:
mymodel = sde.SDE_Markov(mymat=np.array([[0.]]), myvar =np.array([[1.]]),myinit=np.array([1.]),myscale=1.,myterm=term,step_size=step)

In [10]:
repeat_time=10000
test=np.zeros(repeat_time)

In [11]:
for k in range(repeat_time):
    test[k]=mymodel.one_step(init)
np.var(test)

0.0099095040998886576

In [ ]:
term=1
step=

In [14]:
S_k=0
for k in range(100):
    xi=np.random.normal(0,1,1)
    S_k=S_k+xi
y_t=S_t+(t-math.floar(t))

NameError: name 'S_t' is not defined

In [16]:
math.floor(1.1)

1

In [36]:
a=2.3

In [37]:
if type(a) is int:
    print("天才")
else:
    print("うんこ")

うんこ


In [38]:
type(a)

float